In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

# Entradas de login e filial
email ='cadastro@rigarr.com.br'
senha = 'Rigarrdistribuidora@2024'
cd = 'Castas'

# Inicializa a lista para armazenar dados dos pedidos
order_data = []

def login(driver):
    driver.get("https://one.bees.com/order-management")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="signInName"]'))).send_keys(email)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys(senha)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(('xpath', '(//button[@class="c-lhghna"])[2]'))).click()
    WebDriverWait(driver,20).until(EC.element_to_be_clickable(('xpath','//button[@data-testid="chip-delete-icon"]'))).click()



def coletar_dados_pedidos(driver):
    """Coleta números dos pedidos da página atual."""
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pedidos = soup.findAll('tr', class_="c-fKvqNg c-fKvqNg-gsmDXe-rowClickable-true")
    for pedido in pedidos:
        colunas = pedido.findAll('td')
        numero_pedido = colunas[0].get_text(strip=True)  # Pega o número do pedido
        order_data.append(numero_pedido)

def navegar_paginas(driver):
    """Navega por todas as páginas de pedidos."""
    for numero_pagina in range(1, 100):  # Ajuste o limite conforme necessário
        try:
            page_xpath = f'//li[@title="page {numero_pagina}"]'
            coletar_dados_pedidos(driver)
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, page_xpath))
            )
            next_button.click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))) 
        except Exception as e:
            print("Sem mais páginas")
            break

def obter_detalhes_pedido(driver, order):
    """Obtém os detalhes de todos os produtos de um pedido específico."""
    driver.get(f"https://one.bees.com/order-management/closed-orders/{order}")
    time.sleep(5)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]')))
    tbody = driver.find_element(By.XPATH, '//tbody[@role="rowgroup"]')  
    produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')
    detalhes_pedido = driver.find_element(By.CLASS_NAME, 'c-jhfORj')

    total_elemento = driver.find_element(By.XPATH, '//div[@data-testid="order-total-value"]')
    total = total_elemento.find_element(By.TAG_NAME, 'div').text.strip()

    detalhes_produtos = []  # Limpa a lista de produtos para cada pedido

    for produto in produtos:
        nome_sku = produto.find_element(By.XPATH, './/td[1]//p[1]').text.strip()
        sku = produto.find_element(By.XPATH, './/td[1]//p[2]').text.strip()
        quantidade_entregue = produto.find_element(By.XPATH, './/td[2]').text.strip()
        quantidade_nao_entregue = produto.find_element(By.XPATH, './/td[3]').text.strip()
        preco = produto.find_element(By.XPATH, '//div[contains(@data-testid, "product_price_")]/p').text.strip()
        filial = detalhes_pedido.find_element(By.XPATH,'//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[2]/p[2]').text.strip()
        forma_pgto = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[2]/div[3]/p[2]').text.strip()
        endereco_entrega = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[5]/p[2]').text.strip()
        cidade_uf = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[5]/p[3]').text.strip()
        cep = detalhes_pedido.find_element('xpath', '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[5]/p[4]').text.strip()
        coordenadas = detalhes_pedido.find_element('xpath','//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[5]/p[5]').text.strip()
        documento = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[2]/p[2]').text.strip()
        data_pedido = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[2]/div[1]/p[2]').text.strip()
        data_entrega = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[2]/div[2]/p[2]').text.strip()
        status = detalhes_pedido.find_element(By.XPATH,'//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[1]/p[2]').text.strip()
        nome_comercial = detalhes_pedido.find_element(By.XPATH,'//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[7]/div[3]/div[1]/div[3]/p[2]').text.strip()
        detalhes_produtos.append({
            "Número do pedido": order,
            "Status":status,
            "Nome do produto": nome_sku,
            "SKU": sku,
            "Quantidade entregue": quantidade_entregue,
            "Quantidade não entregue": quantidade_nao_entregue,
            "Preço": preco,
            "Forma de Pagamento": forma_pgto,
            "Endereço de Entrega": endereco_entrega,
            "Coordenadas":coordenadas,
            "Cidade/UF":cidade_uf,
            "CEP":cep,
            "Filial":filial,
            "Documento": documento,
            "Total do pedido": total,
            "Data Pedido": data_pedido,
            "Data Entrega": data_entrega,
            "Nome Comercial":nome_comercial
        })

    # Salva todos os produtos deste pedido no CSV de uma vez
    df = pd.DataFrame(detalhes_produtos)
    caminho = f'Pedidos_A_Ser_Entregue_{cd}.csv'
    
    if os.path.exists(caminho):
        df.to_csv(caminho, mode='a', header=False, index=False, encoding='utf-8-sig')
    else:
        df.to_csv(caminho, index=False, encoding='utf-8-sig')

# Inicializa o driver
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=options)
driver.maximize_window()
# Realiza o login
login(driver)

# Lê a planilha existente, se ela existir, e atualiza o conjunto de pedidos existentes
caminho = f'Pedidos_A_Ser_Entregue_{cd}.csv'
pedidos_existentes = set()  # Converte para conjunto
if os.path.exists(caminho):
    df_existente = pd.read_csv(caminho)
    pedidos_existentes.update(df_existente["Número do pedido"].astype(str).to_list())  # Garante que os números são strings

# Navega pelas páginas de pedidos e gera uma lista de pedidos ordenada e sem duplicatas
navegar_paginas(driver)
time.sleep(1)
order_data = sorted(set(order_data) - pedidos_existentes)

# Coleta os detalhes dos pedidos um a um, na ordem crescente
for order in order_data:
    try:
        obter_detalhes_pedido(driver, order)
        time.sleep(1)
        print(f"Detalhes do pedido {order} coletados com sucesso!")
    except Exception as e:
        print(f"Erro ao coletar detalhes do pedido {order}")

# Fecha o navegador
driver.quit()


Sem mais páginas
Detalhes do pedido 9193983338 coletados com sucesso!
Detalhes do pedido 9194112853 coletados com sucesso!
Detalhes do pedido 9194718616 coletados com sucesso!
Detalhes do pedido 9195014570 coletados com sucesso!
Detalhes do pedido 9195567910 coletados com sucesso!
Detalhes do pedido 9195870392 coletados com sucesso!
Detalhes do pedido 9195961390 coletados com sucesso!
Detalhes do pedido 9196233772 coletados com sucesso!
Detalhes do pedido 9196319048 coletados com sucesso!
Detalhes do pedido 9196654869 coletados com sucesso!
Detalhes do pedido 9196802035 coletados com sucesso!
